# Standardize survey data
**Note:** This code results in `../output/survey.sqlite` which should be moved to `$BIODIVERSITY_DATA/survey/` for dependent modules to work.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pandas import ExcelFile

SURVEY_DATA_COLUMNS=['species','latitude','longitude','phase','presence','magnitude']

## Input and output

In [2]:
DB='../output/survey.sqlite'
PHASE_ONE='../../data/field_survey/field_report_2019-06-11_phase1.xlsx'
PHASE_TWO='../../data/field_survey/field_report_2019-08-07_phase2.csv'
PHASE_THREE='../../data/field_survey/field_report_2020-01-03_phase3.xlsx'



## Initiating dataframe
Mandatory column names for database (more can be added): `index,species,latitude,longitude,presence`

In [3]:
surveyData=pd.DataFrame(columns=SURVEY_DATA_COLUMNS)

The function `finalize()` below performs final cleaning up and appending to main dataframe.

In [4]:
def finalize(df,surveyData):
    # Verification
    if df['latitude'].dtype!='float':
        raise TypeError("Column 'latitude' should be float.")
    elif df['longitude'].dtype!='float':
        raise TypeError("Column 'longitude' should be float.")

    surveyData=pd.concat([surveyData,df],sort=True)
    print("After appending to main dataframe: ",surveyData.shape)
    return surveyData

## Phase 1
Processing one sheet at a time. The below function reads in a sheet and modifies columns.
Then, we process each species individually. Note that there are a number of duplicate entries.

In [ ]:
def process_phase_one_sheet(sheetName,speciesName):
    df=pd.read_excel(PHASE_ONE,sheet_name=sheetName)
    df.columns=map(str.lower,df.columns)
    print("Original data shape: ",df.shape)
    
    # Retain required columns
    if speciesName=='Parthenium hysterophorus':
        df=df[['latitude','longitude','magnitude','presence_absence']]
    elif speciesName=='Eichhornia crassipes':
        df=df[['x_coordi','y_coordi','magnitude']]
        df.columns=['latitude','longitude','magnitude']
    else:
        df=df[['latitude','longitude','magnitude']]
    
    # Delete unwanted rows
    df=df.drop_duplicates()
    df=df[df.latitude!='Absent']
    df=df[df.longitude!='Absent']
    print("After dropping duplicates and extra columns: ",df.shape)
    
    # Create presence column
    df['presence']=False
    if speciesName=='Parthenium hysterophorus':
        df.loc[df['presence_absence']=='Presence','presence']=True
        df=df.drop(['presence_absence'],axis=1)
        df.loc[df['magnitude'].isna(),'magnitude']='Not recorded.'     
    else:
        df=df[~df['magnitude'].isna() & ~df['latitude'].isna() & ~df['longitude'].isna()]
        if speciesName=='Ageratina adenophora':
            df.loc[(df['magnitude']!='Absent') & (df['magnitude']!='') & (df['magnitude']!='Not recorded'),'presence']=True
        else:
            df.loc[(df['magnitude']!='Absent') & (df['magnitude']!=''),'presence']=True
    
    # Summary
    print("Final number of points: ",df.shape)
    df['species']=speciesName
    df['phase']=1
    print("Number of presence points:",df[df['presence']==True].shape)
    print("Number of absence points:",df[df['presence']==False].shape)

    return df

### Lantana camara

In [ ]:
df=process_phase_one_sheet('Lentena','Lantana camara')
surveyData=finalize(df,surveyData)
# df=pd.read_excel(PHASE_ONE,sheet_name='Lentena')
# df.columns=map(str.lower,df.columns)
# df['presence']=False
# df

### Chromolaena odorata

In [ ]:
df=process_phase_one_sheet('Chromoleana','Chromolaena odorata')
surveyData=finalize(df,surveyData)

###  Ipomoea carnea

In [ ]:
df=process_phase_one_sheet('Ipomea','Ipomoea carnea')
surveyData=finalize(df,surveyData)

### Mikania micrantha

In [ ]:
df=process_phase_one_sheet('Mikeniea','Mikania micrantha')
surveyData=finalize(df,surveyData)

### Ageratina adenophora

In [ ]:
df=process_phase_one_sheet('Ageratina','Ageratina adenophora')
x=df['longitude'].tolist()
print("Errors in 'longitude'",[i for i in range(len(x)) if type(x[i])==str])
print(x[29],x[519])
df.loc[29,'longitude']=85.0775
df=df.drop(df.index[519])
x=df['longitude'].tolist()
print("Verifying after making corrections: ",[i for i in range(len(x)) if type(x[i])==str])
x=df['latitude'].tolist()
print("No errors in 'latitude'",[i for i in range(len(x)) if type(x[i])==str])
df=df.astype({'latitude':'float','longitude':'float'})
print("Removed 'Not recorded' entries for now.")
print("Number of presence points:",df[df['presence']==True].shape)
print("Number of absence points:",df[df['presence']==False].shape)
surveyData=finalize(df,surveyData)

### Parthenium hysterophorus

In [ ]:
df=process_phase_one_sheet('Parthenium','Parthenium hysterophorus')
surveyData=finalize(df,surveyData)
# df=pd.read_excel(PHASE_ONE,sheet_name='Parthenium')
# df.columns=map(str.lower,df.columns)
# df['presence']=False
# df.loc[df['presence_absence']=='Presence','presence']=True

### Eichhornia crassipes

In [ ]:
df=process_phase_one_sheet('Eichorniea','Eichhornia crassipes')
surveyData=finalize(df,surveyData)

## Phase 2
This is present in a single csv file.

In [ ]:
df=pd.read_csv(PHASE_TWO)
df=df[['longitude','latitude','Species','Field  Validation status']]
df.columns=['longitude','latitude','species','presence']
df['presence_new']=False
df.loc[df['presence']=='Present','presence_new']=True
df=df.drop(['presence'],axis=1)
df=df.rename(columns={"presence_new":"presence"})
df['phase']=2
df.loc[df.species=='Lantana','species']='Lantana camara'
df.loc[df.species=='lantana','species']='Lantana camara'
df.loc[df.species=='Chromoleana','species']='Chromolaena odorata'
df.loc[df.species=='Ipomea','species']='Ipomoea carnea'
df.loc[df.species=='Mikania','species']='Mikania micrantha'
df.loc[df.species=='Ageratina','species']='Ageratina adenophora'
df.loc[df.species=='parthenium','species']='Parthenium hysterophorus'
df.loc[df.species=='Parthenium','species']='Parthenium hysterophorus'
dupl=df[df.species=='Parthenium/Lantana']
dupl.species='Lantana camara'
df.loc[df.species=='Parthenium/Lantana','species']='Parthenium hysterophorus'
df=pd.concat([df,dupl],sort=True)

# Absent columns
dupl=df[(df.species=='Absent') | (df.species=='absent')]
for sp in ['Lantana camara','Chromolaena odorata','Ipomoea carnea','Mikania micrantha','Ageratina adenophora','Parthenium hysterophorus']:
    duplSp=dupl.copy()
    duplSp.species=sp
    df=pd.concat([df,duplSp],sort=True)

df=df[(df.species!='Absent') & (df.species!='absent')]
df.species.unique()
print("Original data shape",df.shape)
df=df.dropna()
print("After dropping rows with NaNs", df.shape)
print("Number of presence points:",df[df['presence']==True].shape)
print("Number of absence points:",df[df['presence']==False].shape)
surveyData=finalize(df,surveyData)

## Push to database
Assumes that the database has the following table:
```
CREATE TABLE "survey" (
	"species"	TEXT,
	"latitude"	REAL,
	"longitude"	REAL,
    "phase"	INTEGER,
	"presence"	INTEGER,
	"magnitude"	TEXT,
	PRIMARY KEY("species","latitude","longitude","phase")
);
```

## Phase 3
Processing one sheet at a time

### Chromolaena

In [5]:
df=pd.read_excel(PHASE_THREE,sheet_name='Chromolaena presence')
df['presence'] = 1
df=df.drop(['S.N', 'elevation'],axis=1)
df=df.rename(columns={"Species":"species"})
df['species']= "Chromolaena odorata"
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)


df=pd.read_excel(PHASE_THREE,sheet_name='Chromolaena absence')
df['presence'] =0
df=df.drop(['S.N','Species(Chromolaena)','elevation'],axis=1)
df['species']= "Chromolaena odorata"
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)



After appending to main dataframe:  (205, 6)
After appending to main dataframe:  (354, 6)


### Lantana Camara

In [6]:
df=pd.read_excel(PHASE_THREE,sheet_name='Lantana presence')
df['presence'] = 1
df=df.drop(['SN'],axis=1)
df['species']= 'Lantana camara'
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)

df=pd.read_excel(PHASE_THREE,sheet_name='Lantana absence')
df['presence'] = 0
df=df.drop(['SN'],axis=1)
df=df.drop(['Species/lantana'],axis=1)
df['species']= 'Lantana camara'
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)




After appending to main dataframe:  (645, 6)
After appending to main dataframe:  (852, 6)


### Mikania

In [7]:
df=pd.read_excel(PHASE_THREE,sheet_name='Mikania presence')
df['presence'] = 1
df=df.drop(['S.N','elevation'],axis=1)
df=df.drop(df.columns[4],axis='columns')
df=df.rename(columns={"Species":"species", "Magnitude":"magnitude"})
df['species']= 'Mikania micrantha'
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)

df=pd.read_excel(PHASE_THREE,sheet_name='Mikania absence')
df['presence'] = 0
df=df.drop(['S.N','elevation'],axis=1)
df=df.rename(columns={"Species/mikania":"species"})
df['species']= 'Mikania micrantha'
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)




After appending to main dataframe:  (970, 6)
After appending to main dataframe:  (1130, 6)


### Ipomea

In [8]:
df=pd.read_excel(PHASE_THREE,sheet_name='Ipomea presence')
df = df[:-1]
df=df.rename(columns={"SN":"species", "Longitude":"longitude"})
df['species'] = 'Ipomoea carnea'
df.drop(df.columns[[3, 5]], axis=1, inplace=True)
df['presence']=1
df['magnitude'] = df['magnitude'].astype(str)
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)




After appending to main dataframe:  (1142, 6)


### Eichornia

In [9]:
df=pd.read_excel(PHASE_THREE,sheet_name='Eichornia presence')
df=df.drop(['S.N','elevation'],axis=1)
df=df.rename(columns={"Species":"species"})
df['magnitude'] = df['magnitude'].astype(str)
df['presence'] =1
df['species'] = 'Eichhornia crassipes'
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)



After appending to main dataframe:  (1148, 6)


### Parthenium

In [10]:
df=pd.read_excel(PHASE_THREE,sheet_name='parthenium_presenceabsence')
df.drop(df.columns[4:11], axis=1, inplace=True)
df=df.drop(['SN','Elevation','Field_visi', 'value'],axis=1)
df=df.rename(columns={"Presence_a":"presence", "Latitude":"latitude", "Longitude":"longitude"})
df.loc[df.presence == "Parthenium present","presence"] = 1
df.loc[df.presence == "Parthenium absent","presence"] = 0
df['species'] = 'Parthenium hysterophorus'
df['phase']=3
df=df.drop_duplicates()
surveyData=finalize(df,surveyData)


After appending to main dataframe:  (1537, 6)


In [ ]:
conn = sqlite3.connect(DB)
cur=conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS "survey" ( \
    "species"    TEXT, \
    "latitude"    REAL, \
    "longitude"    REAL, \
    "phase"    INTEGER, \
    "presence"    INTEGER, \
    "magnitude"    TEXT, \
    PRIMARY KEY("species","latitude","longitude","phase") \
);')
surveyData.to_sql('temporary_table',conn,if_exists='replace')
cur=conn.cursor()
cur.execute('INSERT OR REPLACE INTO survey SELECT ' + ','.join(SURVEY_DATA_COLUMNS) + ' FROM temporary_table;')
cur.execute('DROP TABLE temporary_table;')
conn.commit()
conn.close()

## Verification
### Distinct species list

In [ ]:
conn = sqlite3.connect(DB)
cur=conn.cursor()
cur.execute('SELECT DISTINCT species from survey;')
cur.fetchall()

### Distinct values in 'presence' column

In [ ]:
cur.execute('SELECT DISTINCT presence from survey;')
cur.fetchall()

### Distinct values in 'magnitude' column

In [ ]:
cur.execute('SELECT DISTINCT magnitude from survey;')
cur.fetchall()

### Rows in dataframe and database

In [ ]:
cur.execute('SELECT count(*) FROM survey;')
numRows=cur.fetchall()
print("Number of rows in database: %d" %numRows[0])
print("Number of rows in dataframe: %d" %surveyData.shape[0])

**Note:** The discrepancy in the number of rows in database and dataframe is because there are rows for which 'magnitude' differs.

### Checking presence/absence points for each species

In [ ]:
cur.execute('SELECT species,phase,count(*),sum(presence) AS presence,sum((1-presence)) AS absence FROM survey GROUP BY species,phase;')
res=cur.fetchall()
for ele in res:
    print(ele)

In [ ]:
conn.close()